In [2]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os, re, sys
sys.path.insert(0, '/home/paul/language-ident-from-speech/misc')
from get_results import calculate_f_score
from os.path import join, exists

In [4]:
ad_summary = pd.read_csv(join(os.getcwd(), "results", "ad_all_summary.csv"))

In [6]:
ad_summary.head(1)

,expname,training_languages,excluded_languages,seed,classification_3s,classification_10s,classification_30s,accuracy_3s,accuracy_10s,accuracy_30s,c_primary_3s,c_primary_10s,c_primary_30s
0,ad_all_tr_no_th,all,TH,1,/home/paul/gp-data/ad_all_tr_no_th/exp/results...,/home/paul/gp-data/ad_all_tr_no_th/exp/results...,/home/paul/gp-data/ad_all_tr_no_th/exp/results...,0.648,0.783,0.812,0.462,0.294,0.274


In [7]:
f1_df = pd.DataFrame(columns=["expname", "f1_3s", "f1_10s", "f1_30s"])
n = len(ad_summary)
for i in range(n):
    result_row = [ad_summary["expname"][i]]
    target_lang = ad_summary["excluded_languages"][i]
    for secs in ["3s", "10s", "30s"]:
        col = "classification_" + secs
        class_path = ad_summary[col][i]
        if class_path != "blank":
            prec, recall, f1 = calculate_f_score(class_path, target_lang, beta=1)
            result_row.append(f1)
        else:
            result_row.append("none")
    f1_df.loc[i] = result_row

In [9]:
ad_summary = ad_summary.merge(f1_df, on="expname")

In [10]:
ad_summary.head(2)

,expname,training_languages,excluded_languages,seed,classification_3s,classification_10s,classification_30s,accuracy_3s,accuracy_10s,accuracy_30s,c_primary_3s,c_primary_10s,c_primary_30s,f1_3s,f1_10s,f1_30s
0,ad_all_tr_no_th,all,TH,1,/home/paul/gp-data/ad_all_tr_no_th/exp/results...,/home/paul/gp-data/ad_all_tr_no_th/exp/results...,/home/paul/gp-data/ad_all_tr_no_th/exp/results...,0.648,0.783,0.812,0.462,0.294,0.274,0.763449,0.895131,0.917587
1,ad_all_tr_no_fr,all,FR,1,/home/paul/gp-data/ad_all_tr_no_fr/exp/results...,/home/paul/gp-data/ad_all_tr_no_fr/exp/results...,/home/paul/gp-data/ad_all_tr_no_fr/exp/results...,0.629,0.760,0.786,0.489,0.326,0.308,0.453090,0.627137,0.661213
